In [1]:
import pandas as pd
import numpy as np
from keras.layers import Activation, Dense, Dropout, SpatialDropout1D,Input,Masking,Bidirectional, TimeDistributed
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import LSTM,GRU
from keras.models import Sequential, Model
from keras.preprocessing import sequence
from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping
from random import seed
#seed(1)
import tensorflow as tf
from keras.layers import Input, Dense
from keras.models import Model
import math
from sklearn.metrics import roc_auc_score
import itertools
from sklearn.metrics import classification_report, confusion_matrix
from prettytable import PrettyTable

#defining the early stopping criteria
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1,restore_best_weights=True,patience=3)


Using TensorFlow backend.


In [2]:
gs = pd.read_csv('LSTM_1_hour_2.csv')
gs.drop('Unnamed: 0',axis=1,inplace=True)

In [3]:
cols_to_use = ['uhid','pulserate', 'ecg_resprate',
       'spo2', 'heartrate', 'mean_bp', 'sys_bp', 'dia_bp',
       'peep', 'pip', 'map', 'tidalvol', 'minvol', 'ti', 'fio2',
       'abd_difference_y',
       'abdomen_girth_y','currentdateheight',
       'currentdateweight','dischargestatus', 
       'new_ph', 
       'rbs',  'stool_day_total', 
       'temp', 'total_intake', 'totalparenteralvolume',
       'tpn-tfl', 'typevalue_Antibiotics', 'typevalue_Inotropes',
       'urine', 'urine_per_hour',
       'urine_per_kg_hour','gender', 'birthweight',
       'birthlength', 'birthheadcircumference', 'inout_patient_status',
       'gestationweekbylmp', 'gestationdaysbylmp',
       'baby_type', 'central_temp', 'apgar_onemin', 'apgar_fivemin',
       'apgar_tenmin', 'motherage', 'conception_type', 'mode_of_delivery',
       'steroidname', 'numberofdose', 'gestation']

In [4]:
def range_finder(p):
    length = p
    fractional = (p/15.0) - math.floor(p/15.0)
    return int(round(fractional*15))

In [5]:
def randomize(u):
    
    f_df = pd.DataFrame(columns=u.columns)
    for i in u.uhid.unique():
        x = u[u['uhid']==i]
        x = x[range_finder(len(x)):len(x)]

        f_df = f_df.append(x,ignore_index=True)
        
        return f_df

In [6]:
gd = gs[cols_to_use]

In [7]:
def make_lstm(gd):


    final_df = gd.copy()
    



    final_df.fillna(-999,inplace=True)


    # In[ ]:


    train = final_df[:2520]
    test = final_df[2520:]


    # In[ ]:


    y_train = train['dischargestatus']
    X_train = train.drop('dischargestatus',axis=1)
    X_train = X_train.drop('uhid',axis=1)
    #X_train = X_train.drop('visittime',axis=1)

    y_test = test['dischargestatus']
    X_test = test.drop('dischargestatus',axis=1)
    X_test = X_test.drop('uhid',axis=1)
    #X_test = X_test.drop('startdate',axis=1)


    # In[ ]:

    auc_roc_inter = []
    val_a = []
    train_a = []


    #converting the data into a numpy array
    X_train = np.array(X_train)
    y_train = np.array(y_train)
    X_test = np.array(X_test)
    y_test = np.array(y_test)
    ytrain1 = []
    for i in range(0,len(y_train),15):
        #print(i)
        y1 = y_train[i:i+15]
        ytrain1.append(y1[-1])

    ytest1 = []
    for i in range(0,len(y_test),15):
        #print(i)
        y1 = y_test[i:i+15]
        ytest1.append(y1[-1])

    ytrain1 = np.array(ytrain1)
    ytest1 = np.array(ytest1)

    Xtrain = np.reshape(X_train, (-1, 15, X_train.shape[1]))
    Xtest = np.reshape(X_test, (-1, 15, X_test.shape[1]))

    return Xtrain,Xtest,ytrain1,ytest1

In [8]:
def lstm_model(Xtrain,Xtest,ytrain1,ytest1):
    auc_roc_inter = []
    val_a = []
    train_a = []
    for i in range(2):
        #Building the LSTM model
        X = Input(shape=(None, 48), name='X')
        mX = Masking()(X)
        lstm = Bidirectional(LSTM(units=512,activation='tanh',return_sequences=True,recurrent_dropout=0.5,dropout=0.3))
        mX = lstm(mX)
        L = LSTM(units=64,activation='tanh',return_sequences=False)(mX)
        y = Dense(1, activation="sigmoid")(L)
        outputs = [y]
        inputs = [X]
        model = Model(inputs,outputs)
        model.compile(loss="binary_crossentropy",optimizer='adam',metrics=['accuracy'])

        v_a = []
        t_a = []
        #fitting the model
        model.fit(Xtrain, ytrain1, batch_size=60 ,validation_split=0.15,epochs=38,callbacks=[es])
        #history = model.fit(Xtrain, ytrain1, batch_size=60 ,validation_split=0.15,epochs=38,callbacks=[es])
        for i in range(len(model.history.history['val_accuracy'])):
            v_a.append(model.history.history['val_accuracy'][i])
            t_a.append(model.history.history['accuracy'][i])
        #predictions
        y_pred = model.predict(Xtest)
        #y_pred = y_pred.round()
        y_test = np.array(ytest1)
        y_pred = np.array(y_pred)
        y_test = pd.DataFrame(y_test)
        y_test = np.array(y_test)

        def acc(x):
            if x>0.5:
                return 1
            else:
                return 0

        y_model=[]
        for i in y_pred:
            y_model.append(acc(i))
        y_answer=[]
        for j in y_test:
            y_answer.append(acc(j))
            
        val_a.append(v_a)
        train_a.append(t_a)
        auc_roc_inter.append(roc_auc_score(y_answer,y_pred))
        continue
    
    
        
    return auc_roc_inter,y_model,y_answer

In [9]:
Xtrain,Xtest,ytrain1,ytest1 = make_lstm(gd)

In [10]:
import scipy.stats
def mean_confidence_interval(data, confidence=0.95):
    a = 1.0 * np.array(data)
    n = len(a)
    m, se = np.mean(a), scipy.stats.sem(a)
    h = se * scipy.stats.t.ppf((1 + confidence) / 2., n-1)
    return m, m-h, m+h

In [11]:
an,y_model,y_answer = lstm_model(Xtrain,Xtest,ytrain1,ytest1)

c_a = mean_confidence_interval(an)



Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Train on 142 samples, validate on 26 samples
Epoch 1/38
142/142 [==============================] - 1s 11ms/step - loss: 0.6491 - accuracy: 0.5986 - val_loss: 0.7028 - val_accuracy: 0.5385
Epoch 2/38
142/142 [==============================] - 0s 2ms/step - loss: 0.5245 - accuracy: 0.6831 - val_loss: 0.7359 - val_accuracy: 0.5769
Epoch 3/38
142/142 [==============================] - 0s 2ms/step - loss: 0.4319 - accuracy: 0.7746 - val_loss: 0.3643 - val_accuracy: 0.6923
Epoch 4/38
142/142 [==============================] - 0s 2ms/step - loss: 0.3073 - accuracy: 0.9366 - val_loss: 0.1613 - val_accuracy: 1.0000
Epoch 5/38
142/142 [==============================] - 0s 2ms/step - loss: 0.3126 - accuracy: 0.8732 - val_loss: 0.0855 - val_accuracy: 1.0000
Epoch 6/38
142/142 [==============================] - 0s 2ms/step - loss: 0.2144 - accuracy: 0.9085 - val_loss: 0.2057 - val_accuracy: 0.8462
Epoch 7

In [12]:
cm = confusion_matrix(y_answer,y_model)
ppv_1 = round(float(cm[0][0])/(cm[0][0]+cm[0][1]),2)
npv_1 = round(float(cm[1][1])/(cm[1][0]+cm[1][1]),2)

In [13]:
gs = pd.read_csv('LSTM_6_hour_2.csv')
gs.drop('Unnamed: 0',axis=1,inplace=True)

/home/inicuuser/.local/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2714: DtypeWarning: Columns (14,15,16,24,34,40,42,44,45,46,47,52,54,56) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [14]:
gd = gs[cols_to_use]

In [15]:
Xtrain,Xtest,ytrain1,ytest1 = make_lstm(gd)


In [16]:
an,y_model,y_answer = lstm_model(Xtrain,Xtest,ytrain1,ytest1)

c_6 = mean_confidence_interval(an)


Train on 142 samples, validate on 26 samples
Epoch 1/38
142/142 [==============================] - 3s 21ms/step - loss: 0.6150 - accuracy: 0.6620 - val_loss: 0.2425 - val_accuracy: 1.0000
Epoch 2/38
142/142 [==============================] - 0s 3ms/step - loss: 0.3632 - accuracy: 0.8803 - val_loss: 0.0903 - val_accuracy: 1.0000
Epoch 3/38
142/142 [==============================] - 0s 3ms/step - loss: 0.2525 - accuracy: 0.9507 - val_loss: 0.0443 - val_accuracy: 1.0000
Epoch 4/38
142/142 [==============================] - 0s 3ms/step - loss: 0.1467 - accuracy: 0.9718 - val_loss: 0.0115 - val_accuracy: 1.0000
Epoch 5/38
142/142 [==============================] - 0s 3ms/step - loss: 0.0822 - accuracy: 0.9859 - val_loss: 0.0063 - val_accuracy: 1.0000
Epoch 6/38
142/142 [==============================] - 0s 3ms/step - loss: 0.0389 - accuracy: 1.0000 - val_loss: 0.0038 - val_accuracy: 1.0000
Epoch 7/38
142/142 [==============================] - 0s 3ms/step - loss: 0.0404 - accuracy: 0.9859 - 

In [17]:
cm = confusion_matrix(y_answer,y_model)
ppv_6 = round(float(cm[0][0])/(cm[0][0]+cm[0][1]),2)
npv_6 = round(float(cm[1][1])/(cm[1][0]+cm[1][1]),2)

In [18]:
gs = pd.read_csv('LSTM_12_hour_2.csv')
gs.drop('Unnamed: 0',axis=1,inplace=True)
gd = gs[cols_to_use]
Xtrain,Xtest,ytrain1,ytest1 = make_lstm(gd)
an,y_model,y_answer = lstm_model(Xtrain,Xtest,ytrain1,ytest1)

c_12 = mean_confidence_interval(an)
cm = confusion_matrix(y_answer,y_model)
ppv_12 = round(float(cm[0][0])/(cm[0][0]+cm[0][1]),2)
npv_12 = round(float(cm[1][1])/(cm[1][0]+cm[1][1]),2)

/home/inicuuser/.local/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2714: DtypeWarning: Columns (14,15,16,24,34,40,41,42,44,45,46,47,52,54,56,66,67) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Train on 142 samples, validate on 26 samples
Epoch 1/38
142/142 [==============================] - 4s 26ms/step - loss: 0.6242 - accuracy: 0.6479 - val_loss: 1.1371 - val_accuracy: 0.0000e+00
Epoch 2/38
142/142 [==============================] - 0s 3ms/step - loss: 0.4225 - accuracy: 0.7183 - val_loss: 0.3657 - val_accuracy: 0.9231
Epoch 3/38
142/142 [==============================] - 0s 3ms/step - loss: 0.2288 - accuracy: 0.9225 - val_loss: 0.1287 - val_accuracy: 1.0000
Epoch 4/38
142/142 [==============================] - 0s 3ms/step - loss: 0.1183 - accuracy: 0.9930 - val_loss: 0.0415 - val_accuracy: 1.0000
Epoch 5/38
142/142 [==============================] - 0s 3ms/step - loss: 0.0635 - accuracy: 1.0000 - val_loss: 0.0195 - val_accuracy: 1.0000
Epoch 6/38
142/142 [==============================] - 0s 3ms/step - loss: 0.0193 - accuracy: 1.0000 - val_loss: 0.0097 - val_accuracy: 1.0000
Epoch 7/38
142/142 [==============================] - 0s 3ms/step - loss: 0.0091 - accuracy: 1.000

142/142 [==============================] - 0s 3ms/step - loss: 5.5501e-04 - accuracy: 1.0000 - val_loss: 4.4174e-04 - val_accuracy: 1.0000
Epoch 27/38
142/142 [==============================] - 0s 3ms/step - loss: 5.9469e-04 - accuracy: 1.0000 - val_loss: 4.2162e-04 - val_accuracy: 1.0000
Epoch 28/38
142/142 [==============================] - 0s 3ms/step - loss: 5.8580e-04 - accuracy: 1.0000 - val_loss: 3.8985e-04 - val_accuracy: 1.0000
Epoch 29/38
142/142 [==============================] - 0s 3ms/step - loss: 5.3440e-04 - accuracy: 1.0000 - val_loss: 3.7136e-04 - val_accuracy: 1.0000
Epoch 30/38
142/142 [==============================] - 0s 3ms/step - loss: 8.0894e-04 - accuracy: 1.0000 - val_loss: 3.5882e-04 - val_accuracy: 1.0000
Epoch 31/38
142/142 [==============================] - 0s 3ms/step - loss: 5.7857e-04 - accuracy: 1.0000 - val_loss: 3.3675e-04 - val_accuracy: 1.0000
Epoch 32/38
142/142 [==============================] - 0s 3ms/step - loss: 4.8510e-04 - accuracy: 1.0000 -

In [19]:
gs = pd.read_csv('LSTM_48_hour_2.csv')
gs.drop('Unnamed: 0',axis=1,inplace=True)
gd = gs[cols_to_use]
Xtrain,Xtest,ytrain1,ytest1 = make_lstm(gd)
an,y_model,y_answer = lstm_model(Xtrain,Xtest,ytrain1,ytest1)

c_48 = mean_confidence_interval(an)
cm = confusion_matrix(y_answer,y_model)
ppv_48 = round(float(cm[0][0])/(cm[0][0]+cm[0][1]),2)
npv_48 = round(float(cm[1][1])/(cm[1][0]+cm[1][1]),2)

/home/inicuuser/.local/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2714: DtypeWarning: Columns (14,15,16,24,31,32,34,37,38,40,41,42,43,44,45,46,47,52,54,56,66,67) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Train on 142 samples, validate on 26 samples
Epoch 1/38
142/142 [==============================] - 5s 34ms/step - loss: 0.6342 - accuracy: 0.5352 - val_loss: 0.0968 - val_accuracy: 1.0000
Epoch 2/38
142/142 [==============================] - 1s 5ms/step - loss: 0.4233 - accuracy: 0.7324 - val_loss: 0.0475 - val_accuracy: 1.0000
Epoch 3/38
142/142 [==============================] - 1s 5ms/step - loss: 0.2550 - accuracy: 0.9085 - val_loss: 0.0287 - val_accuracy: 1.0000
Epoch 4/38
142/142 [==============================] - 1s 4ms/step - loss: 0.1434 - accuracy: 1.0000 - val_loss: 0.0125 - val_accuracy: 1.0000
Epoch 5/38
142/142 [==============================] - 1s 5ms/step - loss: 0.0873 - accuracy: 1.0000 - val_loss: 0.0053 - val_accuracy: 1.0000
Epoch 6/38
142/142 [==============================] - 1s 5ms/step - loss: 0.0474 - accuracy: 1.0000 - val_loss: 0.0032 - val_accuracy: 1.0000
Epoch 7/38
142/142 [==============================] - 1s 4ms/step - loss: 0.0188 - accuracy: 1.0000 - 

In [20]:
gs = pd.read_csv('LSTM_1_week_2.csv')
gs.drop('Unnamed: 0',axis=1,inplace=True)
gd = gs[cols_to_use]
Xtrain,Xtest,ytrain1,ytest1 = make_lstm(gd)
an,y_model,y_answer = lstm_model(Xtrain,Xtest,ytrain1,ytest1)

c_1w = mean_confidence_interval(an)
cm = confusion_matrix(y_answer,y_model)
ppv_1w = round(float(cm[0][0])/(cm[0][0]+cm[0][1]),2)
npv_1w = round(float(cm[1][1])/(cm[1][0]+cm[1][1]),2)

Train on 142 samples, validate on 26 samples
Epoch 1/38
142/142 [==============================] - 5s 37ms/step - loss: 0.5924 - accuracy: 0.6831 - val_loss: 0.0604 - val_accuracy: 1.0000
Epoch 2/38
142/142 [==============================] - 1s 4ms/step - loss: 0.3967 - accuracy: 0.7465 - val_loss: 0.0332 - val_accuracy: 1.0000
Epoch 3/38
142/142 [==============================] - 1s 5ms/step - loss: 0.2074 - accuracy: 0.9225 - val_loss: 0.0248 - val_accuracy: 1.0000
Epoch 4/38
142/142 [==============================] - 1s 5ms/step - loss: 0.1009 - accuracy: 1.0000 - val_loss: 0.0130 - val_accuracy: 1.0000
Epoch 5/38
142/142 [==============================] - 1s 5ms/step - loss: 0.0533 - accuracy: 1.0000 - val_loss: 0.0044 - val_accuracy: 1.0000
Epoch 6/38
142/142 [==============================] - 1s 5ms/step - loss: 0.0361 - accuracy: 0.9930 - val_loss: 0.0018 - val_accuracy: 1.0000
Epoch 7/38
142/142 [==============================] - 1s 4ms/step - loss: 0.0155 - accuracy: 1.0000 - 

In [21]:
gs = pd.read_csv('LSTM_2_week.csv')
gs.drop('Unnamed: 0',axis=1,inplace=True)
gd = gs[cols_to_use]
Xtrain,Xtest,ytrain1,ytest1 = make_lstm(gd)
an,y_model,y_answer = lstm_model(Xtrain,Xtest,ytrain1,ytest1)

c_2w = mean_confidence_interval(an)
cm = confusion_matrix(y_answer,y_model)
ppv_2w = round(float(cm[0][0])/(cm[0][0]+cm[0][1]),2)
npv_2w = round(float(cm[1][1])/(cm[1][0]+cm[1][1]),2)

Train on 142 samples, validate on 26 samples
Epoch 1/38
142/142 [==============================] - 6s 41ms/step - loss: 0.5769 - accuracy: 0.6479 - val_loss: 0.0469 - val_accuracy: 1.0000
Epoch 2/38
142/142 [==============================] - 1s 5ms/step - loss: 0.2974 - accuracy: 0.8451 - val_loss: 0.0215 - val_accuracy: 1.0000
Epoch 3/38
142/142 [==============================] - 1s 5ms/step - loss: 0.1281 - accuracy: 0.9789 - val_loss: 0.0070 - val_accuracy: 1.0000
Epoch 4/38
142/142 [==============================] - 1s 5ms/step - loss: 0.0524 - accuracy: 1.0000 - val_loss: 0.0020 - val_accuracy: 1.0000
Epoch 5/38
142/142 [==============================] - 1s 4ms/step - loss: 0.0205 - accuracy: 1.0000 - val_loss: 8.0677e-04 - val_accuracy: 1.0000
Epoch 6/38
142/142 [==============================] - 1s 5ms/step - loss: 0.0088 - accuracy: 1.0000 - val_loss: 3.9767e-04 - val_accuracy: 1.0000
Epoch 7/38
142/142 [==============================] - 1s 5ms/step - loss: 0.0099 - accuracy: 1

Epoch 18/38
142/142 [==============================] - 1s 5ms/step - loss: 8.7522e-04 - accuracy: 1.0000 - val_loss: 2.9012e-04 - val_accuracy: 1.0000
Epoch 19/38
142/142 [==============================] - 1s 5ms/step - loss: 9.9898e-04 - accuracy: 1.0000 - val_loss: 2.7943e-04 - val_accuracy: 1.0000
Epoch 20/38
142/142 [==============================] - 1s 5ms/step - loss: 7.9520e-04 - accuracy: 1.0000 - val_loss: 2.6952e-04 - val_accuracy: 1.0000
Epoch 21/38
142/142 [==============================] - 1s 5ms/step - loss: 7.4522e-04 - accuracy: 1.0000 - val_loss: 2.6051e-04 - val_accuracy: 1.0000
Epoch 22/38
142/142 [==============================] - 1s 5ms/step - loss: 0.0049 - accuracy: 1.0000 - val_loss: 2.4698e-04 - val_accuracy: 1.0000
Epoch 23/38
142/142 [==============================] - 1s 4ms/step - loss: 0.0029 - accuracy: 1.0000 - val_loss: 2.3262e-04 - val_accuracy: 1.0000
Epoch 24/38
142/142 [==============================] - 1s 4ms/step - loss: 7.0179e-04 - accuracy: 1.00

In [22]:
gs = pd.read_csv('LSTM_3_week_2.csv')
gs.drop('Unnamed: 0',axis=1,inplace=True)
gd = gs[cols_to_use]
Xtrain,Xtest,ytrain1,ytest1 = make_lstm(gd)
an,y_model,y_answer = lstm_model(Xtrain,Xtest,ytrain1,ytest1)

c_3w = mean_confidence_interval(an)
cm = confusion_matrix(y_answer,y_model)
ppv_3w = round(float(cm[0][0])/(cm[0][0]+cm[0][1]),2)
npv_3w = round(float(cm[1][1])/(cm[1][0]+cm[1][1]),2)

Train on 142 samples, validate on 26 samples
Epoch 1/38
142/142 [==============================] - 6s 44ms/step - loss: 0.5537 - accuracy: 0.7254 - val_loss: 0.0719 - val_accuracy: 1.0000
Epoch 2/38
142/142 [==============================] - 1s 5ms/step - loss: 0.3324 - accuracy: 0.8310 - val_loss: 0.0353 - val_accuracy: 1.0000
Epoch 3/38
142/142 [==============================] - 1s 5ms/step - loss: 0.1925 - accuracy: 0.9577 - val_loss: 0.0216 - val_accuracy: 1.0000
Epoch 4/38
142/142 [==============================] - 1s 5ms/step - loss: 0.1015 - accuracy: 0.9930 - val_loss: 0.0058 - val_accuracy: 1.0000
Epoch 5/38
142/142 [==============================] - 1s 4ms/step - loss: 0.0366 - accuracy: 1.0000 - val_loss: 0.0028 - val_accuracy: 1.0000
Epoch 6/38
142/142 [==============================] - 1s 5ms/step - loss: 0.0198 - accuracy: 1.0000 - val_loss: 0.0018 - val_accuracy: 1.0000
Epoch 7/38
142/142 [==============================] - 1s 5ms/step - loss: 0.0131 - accuracy: 1.0000 - 

In [23]:
gs = pd.read_csv('LSTM_4_week_2.csv')
gs.drop('Unnamed: 0',axis=1,inplace=True)
gd = gs[cols_to_use]
Xtrain,Xtest,ytrain1,ytest1 = make_lstm(gd)
an,y_model,y_answer = lstm_model(Xtrain,Xtest,ytrain1,ytest1)

c_4w = mean_confidence_interval(an)
cm = confusion_matrix(y_answer,y_model)
ppv_4w = round(float(cm[0][0])/(cm[0][0]+cm[0][1]),2)
npv_4w = round(float(cm[1][1])/(cm[1][0]+cm[1][1]),2)

Train on 142 samples, validate on 26 samples
Epoch 1/38
142/142 [==============================] - 6s 45ms/step - loss: 0.6347 - accuracy: 0.5986 - val_loss: 0.0355 - val_accuracy: 1.0000
Epoch 2/38
142/142 [==============================] - 1s 5ms/step - loss: 0.4210 - accuracy: 0.7254 - val_loss: 0.0221 - val_accuracy: 1.0000
Epoch 3/38
142/142 [==============================] - 1s 5ms/step - loss: 0.2604 - accuracy: 0.8521 - val_loss: 0.0163 - val_accuracy: 1.0000
Epoch 4/38
142/142 [==============================] - 1s 5ms/step - loss: 0.1461 - accuracy: 1.0000 - val_loss: 0.0099 - val_accuracy: 1.0000
Epoch 5/38
142/142 [==============================] - 1s 4ms/step - loss: 0.0904 - accuracy: 0.9930 - val_loss: 0.0042 - val_accuracy: 1.0000
Epoch 6/38
142/142 [==============================] - 1s 4ms/step - loss: 0.0358 - accuracy: 1.0000 - val_loss: 0.0021 - val_accuracy: 1.0000
Epoch 7/38
142/142 [==============================] - 1s 4ms/step - loss: 0.0186 - accuracy: 1.0000 - 

In [24]:
l = [["1 Hour" ,c_a, ppv_1,npv_1],["6 Hour" ,c_6, ppv_6,npv_6],["12 Hour" ,c_12, ppv_12,npv_12],["48 Hour" ,c_48, ppv_48,npv_48],["1 Week" ,c_1w, ppv_1w,npv_1w],["2 Week" ,c_2w, ppv_2w,npv_3w],["3 Week" ,c_3w, ppv_3w,npv_3w],["4 Week" ,c_4w, ppv_4w,npv_4w]]

table = PrettyTable(['Parameter', 'AUC-ROC', 'PPV','NPV'])

for rec in l:
    table.add_row(rec)

print(table)

+-----------+-----------------------------------------------------------------+------+------+
| Parameter |                             AUC-ROC                             | PPV  | NPV  |
+-----------+-----------------------------------------------------------------+------+------+
|   1 Hour  | (0.7824561403508772, -0.005179942726199371, 1.5700922234279537) | 0.86 | 0.6  |
|   6 Hour  |   (0.7977725409836065, 0.6506357316279141, 0.9449093503392989)  | 0.7  | 0.69 |
|  12 Hour  |   (0.7990888650244983, 0.7055292031701859, 0.8926485268788107)  | 0.85 | 0.5  |
|  48 Hour  |   (0.8167186495187854, 0.6715589891132727, 0.961878309924298)   | 0.92 | 0.4  |
|   1 Week  |   (0.8329340892986358, 0.7401633713291447, 0.9257048072681269)  | 0.9  | 0.26 |
|   2 Week  |   (0.8102119976814529, 0.7631923096766999, 0.8572316856862059)  | 0.89 | 0.44 |
|   3 Week  |  (0.7561325674434534, 0.26989195250226966, 1.2423731823846371)  | 0.81 | 0.44 |
|   4 Week  |   (0.7297904929777654, 0.5806701355909867, 0.8